<a href="https://colab.research.google.com/github/alxmarqs/LLMtopics/blob/main/1_4_finetune_marco_aurelio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 4 — Fine-tuning de Decoder Pré-treinado
## Dataset: Frases de Marco Aurélio

## Objetivo

Realizar **fine-tuning** de um modelo decoder pré-treinado (GPT-2) usando **100 frases filosóficas de Marco Aurélio**.

## O que é Fine-tuning?

Fine-tuning é o processo de ajustar um modelo pré-treinado para um domínio ou tarefa específica:
- Modelo já foi treinado em grande corpus (ex: GPT-2 treinado em WebText)
- Ajustamos os pesos usando corpus menor e específico (100 frases filosóficas)
- Modelo aprende vocabulário e estilo do novo domínio

## Características do Dataset

- **100 frases** de Marco Aurélio (Meditações)
- Temática: filosofia estoica, sabedoria, virtude, natureza
- Vocabulário: reflexivo, introspectivo, contemplativo
- Estrutura: aforismos, máximas, reflexões curtas

## Passos do notebook

1. Preparar dataset de frases
2. Carregar tokenizador e modelo base (GPT-2)
3. Tokenizar dataset
4. Configurar treinamento
5. Realizar fine-tuning
6. Comparar modelo base vs modelo ajustado

## 1. Instalação de dependências

In [69]:
!pip install datasets transformers accelerate -q

In [70]:
import torch
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Dispositivo: {device}')

Dispositivo: cuda


---
## 2. Preparar dataset de frases de Marco Aurélio

In [71]:
# Dataset: 100 frases de Marco Aurélio
frases_marco_aurelio = [
    "A felicidade da sua vida depende da qualidade dos seus pensamentos.",
    "Não perca mais tempo discutindo sobre o que um bom homem deve ser. Seja um.",
    "Você tem poder sobre sua mente, não sobre eventos externos. Perceba isso e encontrará a força.",
    "Tudo o que ouvimos é uma opinião, não um fato.",
    "Tudo o que vemos é uma perspectiva, não a verdade.",
    "A alma é tingida pela cor dos seus pensamentos.",
    "A melhor vingança é ser diferente de quem causou o dano.",
    "Aceite as coisas às quais o destino o ata e ame as pessoas com quem o destino o une.",
    "Faça cada coisa na vida como se fosse a última.",
    "A morte sorri para todos nós; tudo o que um homem pode fazer é sorrir de volta.",
    "O que não é bom para a colmeia também não é bom para a abelha.",
    "A perda nada mais é do que mudança, e a mudança é o deleite da natureza.",
    "Se algo é possível para qualquer homem, considere que também é possível para você.",
    "A vida de um homem é o que seus pensamentos fazem dela.",
    "Olhe para dentro. Dentro está a fonte do bem, e ela sempre jorrará se você sempre cavar.",
    "Aquele que vive em harmonia consigo mesmo vive em harmonia com o universo.",
    "Muitas vezes me perguntei como é que todo homem se ama mais do que a todos os outros homens, mas dá menos valor à sua própria opinião do que à dos outros.",
    "A arte de viver assemelha-se mais à luta do que à dança.",
    "O universo é mudança; a vida é opinião.",
    "Não aja como se fosse viver dez mil anos. A morte paira sobre você.",
    "Enquanto viver e enquanto puder, seja bom.",
    "Quão ridículo e quão estranho é surpreender-se com qualquer coisa que acontece na vida.",
    "O objetivo da vida não é estar do lado da maioria, mas escapar de se encontrar nas fileiras dos loucos.",
    "O homem deve ter medo não da morte, mas de nunca começar a viver.",
    "A pobreza é a mãe do crime.",
    "Lembre-se de que tudo o que acontece, acontece como deveria.",
    "Nada acontece a homem algum que ele não seja formado pela natureza para suportar.",
    "A rejeição é a sua própria defesa.",
    "A primeira regra é manter o espírito tranquilo.",
    "A segunda regra é olhar as coisas de frente e saber o que elas são.",
    "Não se deixe levar pelo futuro. Você o encontrará, se necessário, com as mesmas armas da razão que hoje o armam contra o presente.",
    "Sempre que estiver prestes a apontar um defeito em outra pessoa, faça a seguinte pergunta: Que defeito em mim se assemelha ao que estou prestes a criticar?",
    "A vida não é boa nem má, mas apenas o lugar para o bem e o mal.",
    "Se não for certo, não faça; se não for verdade, não diga.",
    "Concentre-se a cada minuto como um romano e um homem.",
    "Faça o que tem à sua frente com seriedade precisa e genuína, ternura, vontade e justiça.",
    "Liberte-se de todas as outras distrações.",
    "A brevidade da vida é a única certeza.",
    "Em breve você terá esquecido tudo; em breve todos terão esquecido você.",
    "O tempo é um rio, um fluxo violento de eventos.",
    "Assim que algo é avistado, é arrastado e outra coisa toma o seu lugar.",
    "Pense na beleza da vida. Observe as estrelas e veja-se correndo com elas.",
    "Tudo o que é belo em si mesmo é completo em si mesmo e não precisa de elogios.",
    "O elogio não torna nada melhor ou pior.",
    "Se você está angustiado por qualquer coisa externa, a dor não se deve à coisa em si, mas à sua estimativa dela.",
    "Você tem o poder de revogar sua estimativa a qualquer momento.",
    "Seja como o promontório contra o qual as ondas quebram continuamente.",
    "Permaneça firme e dome a fúria da água ao seu redor.",
    "Eu sou um homem de sorte, pois, embora isso tenha acontecido comigo, continuo livre de tristeza.",
    "Não fui ferido pelo presente nem tenho medo do futuro.",
    "Isso poderia ter acontecido com qualquer um, mas nem todos teriam permanecido livres de tristeza.",
    "Adapte-se ao ambiente em que sua sorte o colocou.",
    "Ame verdadeiramente os companheiros com quem o destino o ordenou caminhar.",
    "Olhe para o passado, para os impérios em ascensão e queda, e você poderá prever o futuro.",
    "Não há nada de novo sob o sol.",
    "Nenhuma pessoa tem o poder de ter tudo o que deseja.",
    "Está em seu poder não querer o que você não tem e fazer bom uso do que você tem.",
    "A tranquilidade não é nada além da boa ordenação da mente.",
    "O mundo é uma cidade.",
    "Para viver feliz, basta muito pouco.",
    "Tudo está interligado como uma teia sagrada.",
    "Nenhuma coisa é estranha à outra.",
    "Os homens existem para o bem uns dos outros.",
    "Ensine-os ou suporte-os.",
    "A injustiça é uma impiedade.",
    "Aquele que comete injustiça, comete injustiça contra si mesmo.",
    "Muitas vezes peca quem deixa de fazer algo, não apenas quem faz algo.",
    "Comece cada dia dizendo a si mesmo: Hoje encontrarei interferência, ingratidão, insolência, deslealdade, má vontade e egoísmo.",
    "Eles devem isso à ignorância do que é bom ou mau.",
    "Mas eu vi a beleza do bem e a feiura do mal.",
    "Reconheço que o malfeitor tem uma natureza semelhante à minha.",
    "Não do mesmo sangue ou nascimento, mas da mesma mente.",
    "Ninguém pode me prejudicar, pois ninguém pode me envolver em baixeza.",
    "Nascemos para trabalhar juntos como pés, mãos e olhos.",
    "Obstruir uns aos outros é antinatural.",
    "Sentir raiva e virar as costas para alguém é obstrução.",
    "O que morre não cai fora do mundo.",
    "Se fica aqui, muda aqui e se dissolve em suas partes apropriadas.",
    "Essas partes são elementos do universo e de você.",
    "A mudança não é um mal, assim como a persistência no novo estado não é um bem.",
    "Lembre-se de quanto tempo você adiou essas coisas.",
    "Quantas vezes você recebeu uma oportunidade dos deuses e não a usou.",
    "Você deve finalmente perceber de que tipo de mundo você é uma parte.",
    "Existe um limite para o seu tempo.",
    "Se você não usar o tempo para clarear as nuvens, ele passará e você passará.",
    "O tempo nunca retornará.",
    "Descarte sua sede de livros, para que não morra murmurando.",
    "Morra verdadeiramente resignado e grato de coração aos deuses.",
    "Considere continuamente como todas as coisas que existem agora existiram no passado.",
    "Considere que todas as coisas existirão novamente no futuro.",
    "Dramas e cenas inteiras são sempre os mesmos, apenas os atores mudam.",
    "Não despreze a morte, mas aceite-a de bom grado.",
    "A morte é uma das coisas que a natureza deseja.",
    "Busque a sabedoria, não o conhecimento.",
    "O conhecimento é o acúmulo de fatos, a sabedoria é a simplificação.",
    "A melhor maneira de se vingar de um inimigo é não se assemelhar a ele.",
    "Onde um homem pode viver, ele também pode viver bem.",
    "É possível viver bem em um palácio? Sim.",
    "Então é possível viver bem em qualquer lugar.",
    "Olhe sob a superfície; não deixe que a qualidade ou o valor de nada lhe escape."
]

print(f"Total de frases: {len(frases_marco_aurelio)}")
print("\nAmostras:")
for i, frase in enumerate(frases_marco_aurelio[:5], 1):
    print(f"  {i}. {frase}")

Total de frases: 100

Amostras:
  1. A felicidade da sua vida depende da qualidade dos seus pensamentos.
  2. Não perca mais tempo discutindo sobre o que um bom homem deve ser. Seja um.
  3. Você tem poder sobre sua mente, não sobre eventos externos. Perceba isso e encontrará a força.
  4. Tudo o que ouvimos é uma opinião, não um fato.
  5. Tudo o que vemos é uma perspectiva, não a verdade.


In [72]:
# Criar dataset HuggingFace
ds = Dataset.from_dict({"text": frases_marco_aurelio})

# Split train/test (90/10)
ds = ds.train_test_split(test_size=0.1, seed=42)

print(f"Dataset de treino: {len(ds['train'])} frases")
print(f"Dataset de teste: {len(ds['test'])} frases")
print(f"\nExemplo de treino:\n  {ds['train'][0]['text']}")
print(f"\nExemplo de teste:\n  {ds['test'][0]['text']}")

Dataset de treino: 90 frases
Dataset de teste: 10 frases

Exemplo de treino:
  Você tem poder sobre sua mente, não sobre eventos externos. Perceba isso e encontrará a força.

Exemplo de teste:
  Para viver feliz, basta muito pouco.


---
## 3. Preparar tokenizador e modelo base

In [73]:
# Carregar tokenizador
model_name = "pierreguillou/gpt2-small-portuguese"  # GPT-2 base (124M parâmetros)
tok = AutoTokenizer.from_pretrained(model_name)

# GPT-2 não tem pad_token, usamos eos_token
tok.pad_token = tok.eos_token

print(f"Tokenizador: {model_name}")
print(f"Vocabulário: {len(tok)} tokens")
print(f"\nTokens especiais:")
print(f"  EOS: '{tok.eos_token}' (id={tok.eos_token_id})")
print(f"  PAD: '{tok.pad_token}' (id={tok.pad_token_id})")

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

Tokenizador: pierreguillou/gpt2-small-portuguese
Vocabulário: 50257 tokens

Tokens especiais:
  EOS: '<|endoftext|>' (id=0)
  PAD: '<|endoftext|>' (id=0)


In [74]:
# Testar tokenização
exemplo = frases_marco_aurelio[0]
tokens = tok(exemplo)

print(f"Texto: {exemplo}")
print(f"\nTotal de tokens: {len(tokens['input_ids'])}")
print(f"\nTokens IDs: {tokens['input_ids']}")
print(f"\nTokens decodificados:")
for i, token_id in enumerate(tokens['input_ids'][:10]):
    token_str = tok.decode([token_id])
    print(f"  {i}: {token_id:5d} -> '{token_str}'")
print("  ...")

Texto: A felicidade da sua vida depende da qualidade dos seus pensamentos.

Total de tokens: 12

Tokens IDs: [33, 15835, 305, 450, 1172, 8966, 305, 3459, 420, 660, 16573, 14]

Tokens decodificados:
  0:    33 -> 'A'
  1: 15835 -> ' felicidade'
  2:   305 -> ' da'
  3:   450 -> ' sua'
  4:  1172 -> ' vida'
  5:  8966 -> ' depende'
  6:   305 -> ' da'
  7:  3459 -> ' qualidade'
  8:   420 -> ' dos'
  9:   660 -> ' seus'
  ...


---
## 4. Tokenizar dataset completo

In [75]:
def tokenize_function(batch):
    """Tokeniza batch de textos com padding e truncation."""
    return tok(
        batch["text"],
        truncation=True,
        max_length=128,
        padding="max_length"
    )

# Aplicar tokenização
lm_ds = ds.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
    desc="Tokenizando dataset"
)

# Configurar formato PyTorch
lm_ds.set_format(type="torch", columns=["input_ids", "attention_mask"])

print(f"Dataset tokenizado:")
print(f"  Train: {len(lm_ds['train'])} exemplos")
print(f"  Test: {len(lm_ds['test'])} exemplos")
print(f"\nExemplo tokenizado (shape):")
print(f"  input_ids: {lm_ds['train'][0]['input_ids'].shape}")
print(f"  attention_mask: {lm_ds['train'][0]['attention_mask'].shape}")

Tokenizando dataset:   0%|          | 0/90 [00:00<?, ? examples/s]

Tokenizando dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset tokenizado:
  Train: 90 exemplos
  Test: 10 exemplos

Exemplo tokenizado (shape):
  input_ids: torch.Size([128])
  attention_mask: torch.Size([128])


---
## 5. Carregar modelos (base e para fine-tuning)

In [76]:
# Modelo base (para comparação - não treinaremos este)
base_model = AutoModelForCausalLM.from_pretrained(model_name)
base_model.resize_token_embeddings(len(tok))
base_model.to(device)
base_model.eval()

# Modelo que será ajustado
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tok))
model.to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Modelo: {model_name}")
print(f"Total de parâmetros: {total_params:,}")
print(f"Parâmetros treináveis: {trainable_params:,}")
print(f"Dispositivo: {device}")

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/149 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie transformer.wte.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
GPT2LMHeadModel LOAD REPORT from: pierreguillou/gpt2-small-portuguese
Key                                     | Status     |  | 
----------------------------------------+------------+--+-
transformer.h.{0...11}.attn.masked_bias | UNEXPECTED |  | 
transformer.h.{0...11}.attn.bias        | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loading weights:   0%|          | 0/149 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie transformer.wte.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
GPT2LMHeadModel LOAD REPORT from: pierreguillou/gpt2-small-portuguese
Key                                     | Status     |  | 
----------------------------------------+------------+--+-
transformer.h.{0...11}.attn.masked_bias | UNEXPECTED |  | 
transformer.h.{0...11}.attn.bias        | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Modelo: pierreguillou/gpt2-small-portuguese
Total de parâmetros: 124,439,808
Parâmetros treináveis: 124,439,808
Dispositivo: cuda


---
## 6. Inferência com modelo BASE (antes do fine-tuning)

In [77]:
def generate_text(model, prompt, max_new_tokens=30, temperature=0.7, top_k=50, top_p=0.9):
    """Gera texto a partir de um prompt usando o modelo."""
    inputs = tok(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            pad_token_id=tok.eos_token_id
        )

    return tok.decode(outputs[0], skip_special_tokens=True)


# Prompts de teste (no estilo estoico)
test_prompts = [
    "A sabedoria está em",
    "O homem virtuoso",
    "A natureza nos ensina que"
]

print("Inferência com MODELO BASE (sem fine-tuning)")
print("=" * 70)

base_outputs = []
for prompt in test_prompts:
    output = generate_text(base_model, prompt, max_new_tokens=25)
    base_outputs.append(output)
    print(f"\nPrompt: {prompt}")
    print(f"Output: {output}")
    print("-" * 70)

Inferência com MODELO BASE (sem fine-tuning)

Prompt: A sabedoria está em
Output: A sabedoria está em sua forma de uma forma de vida e uma forma de arte. A magia tem suas raízes na sabedoria e a magia está envolvida
----------------------------------------------------------------------

Prompt: O homem virtuoso
Output: O homem virtuoso, e o homem sem vaidade.

"The Lady Who Shaped My Heart" foi lançado em 18 de outubro
----------------------------------------------------------------------

Prompt: A natureza nos ensina que
Output: A natureza nos ensina que o homem é o ser que é dotado de todas as virtudes, mas o homem é o ser que está na forma de ser
----------------------------------------------------------------------


---
## 7. Configurar treinamento

**Nota**: Com apenas 90 frases de treino, usaremos mais épocas para garantir aprendizado.

In [78]:
# Data collator para linguagem causal
collator = DataCollatorForLanguageModeling(
    tokenizer=tok,
    mlm=False  # False = causal LM
)

# Argumentos de treinamento
args = TrainingArguments(
    output_dir="./ft_gpt2_marco_aurelio",

    # Batch sizes (pequeno devido ao dataset pequeno)
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,

    # Mais épocas devido ao dataset pequeno
    num_train_epochs=10,
    learning_rate=5e-6,

    # Logging frequente
    logging_steps=10,
    eval_strategy="epoch",

    # Salvamento
    save_strategy="epoch",
    save_total_limit=2,

    # Otimizações
    fp16=torch.cuda.is_available(),

    # Não reportar para wandb/tensorboard
    report_to="none",

    # Reproducibilidade
    seed=42
)

print("Configurações de treinamento:")
print(f"  Épocas: {args.num_train_epochs}")
print(f"  Batch size (treino): {args.per_device_train_batch_size}")
print(f"  Learning rate: {args.learning_rate}")
print(f"  Mixed precision (fp16): {args.fp16}")
print(f"  Logging a cada: {args.logging_steps} steps")

Configurações de treinamento:
  Épocas: 10
  Batch size (treino): 2
  Learning rate: 5e-06
  Mixed precision (fp16): True
  Logging a cada: 10 steps


In [79]:
# Criar Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=lm_ds["train"],
    eval_dataset=lm_ds["test"],
    data_collator=collator
)

print("Trainer criado com sucesso!")
print(f"Total de steps por época: {len(trainer.get_train_dataloader())}")
print(f"Total de steps no treinamento: {len(trainer.get_train_dataloader()) * args.num_train_epochs}")

Trainer criado com sucesso!
Total de steps por época: 45
Total de steps no treinamento: 450


---
## 8. Executar fine-tuning

In [80]:
print("Iniciando fine-tuning...")
print("=" * 70)

train_result = trainer.train()

print("\n" + "=" * 70)
print("Fine-tuning concluído!")
print("=" * 70)

# Métricas finais
metrics = train_result.metrics
print(f"\nMétricas de treinamento:")
print(f"  Loss final: {metrics['train_loss']:.4f}")
print(f"  Tempo total: {metrics['train_runtime']:.2f}s")
print(f"  Samples/segundo: {metrics['train_samples_per_second']:.2f}")

Iniciando fine-tuning...


Epoch,Training Loss,Validation Loss
1,4.003091,3.759175
2,3.915796,3.671092
3,3.667436,3.628544
4,3.641320,3.603546
5,3.328884,3.593869
6,3.417381,3.581804
7,3.395713,3.575743
8,3.480415,3.573675
9,3.311596,3.571667
10,3.531289,3.570816


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]


Fine-tuning concluído!

Métricas de treinamento:
  Loss final: 3.5650
  Tempo total: 263.12s
  Samples/segundo: 3.42


---
## 9. Avaliar perplexidade no conjunto de teste

In [81]:
# Avaliar modelo ajustado
eval_results = trainer.evaluate()

print("Avaliação no conjunto de teste:")
print("=" * 70)
print(f"  Eval Loss: {eval_results['eval_loss']:.4f}")
print(f"  Perplexidade: {np.exp(eval_results['eval_loss']):.2f}")
print(f"  Runtime: {eval_results['eval_runtime']:.2f}s")

Avaliação no conjunto de teste:
  Eval Loss: 3.5708
  Perplexidade: 35.55
  Runtime: 0.10s


---
## 10. Inferência com modelo AJUSTADO (após fine-tuning)

In [82]:
print("Inferência com MODELO AJUSTADO (após fine-tuning)")
print("=" * 70)

finetuned_outputs = []
for prompt in test_prompts:
    output = generate_text(model, prompt, max_new_tokens=25)
    finetuned_outputs.append(output)
    print(f"\nPrompt: {prompt}")
    print(f"Output: {output}")
    print("-" * 70)

Inferência com MODELO AJUSTADO (após fine-tuning)

Prompt: A sabedoria está em
Output: A sabedoria está em seus limites. Quando você está prestes a aceitar o que você não pode aceitar, você deve aceitar tudo o que você sabe.
----------------------------------------------------------------------

Prompt: O homem virtuoso
Output: O homem virtuoso e virtuoso é bom.

Em muitos mundos, a bondade é a fonte de todas as maldade.


----------------------------------------------------------------------

Prompt: A natureza nos ensina que
Output: A natureza nos ensina que a razão é a razão, a razão é o conhecimento. A razão é a razão. A razão é a razão. A
----------------------------------------------------------------------


---
## 11. Comparação lado a lado: BASE vs AJUSTADO

In [83]:
print("COMPARAÇÃO: MODELO BASE vs MODELO AJUSTADO")
print("=" * 70)

for i, prompt in enumerate(test_prompts):
    print(f"\n[Prompt {i+1}] {prompt}")
    print("-" * 70)
    print(f"\n[BASE]")
    print(f"{base_outputs[i]}")
    print(f"\n[AJUSTADO]")
    print(f"{finetuned_outputs[i]}")
    print("\n" + "=" * 70)

COMPARAÇÃO: MODELO BASE vs MODELO AJUSTADO

[Prompt 1] A sabedoria está em
----------------------------------------------------------------------

[BASE]
A sabedoria está em sua forma de uma forma de vida e uma forma de arte. A magia tem suas raízes na sabedoria e a magia está envolvida

[AJUSTADO]
A sabedoria está em seus limites. Quando você está prestes a aceitar o que você não pode aceitar, você deve aceitar tudo o que você sabe.


[Prompt 2] O homem virtuoso
----------------------------------------------------------------------

[BASE]
O homem virtuoso, e o homem sem vaidade.

"The Lady Who Shaped My Heart" foi lançado em 18 de outubro

[AJUSTADO]
O homem virtuoso e virtuoso é bom.

Em muitos mundos, a bondade é a fonte de todas as maldade.




[Prompt 3] A natureza nos ensina que
----------------------------------------------------------------------

[BASE]
A natureza nos ensina que o homem é o ser que é dotado de todas as virtudes, mas o homem é o ser que está na forma de ser

[

---
## 12. Teste com frases do dataset de treino

In [84]:
print("Completando frases do dataset de treino")
print("=" * 70)

# Pegar primeiros tokens de algumas frases como prompt
for i in range(min(5, len(frases_marco_aurelio))):
    frase = frases_marco_aurelio[i]

    # Criar prompt com primeiras palavras
    words = frase.split()[:5]
    prompt = " ".join(words)

    # Gerar com modelo ajustado
    generated = generate_text(model, prompt, max_new_tokens=20)

    print(f"\n[Frase {i+1}]")
    print(f"Prompt: {prompt}")
    print(f"Original: {frase}")
    print(f"Gerado: {generated}")
    print("-" * 70)

Completando frases do dataset de treino

[Frase 1]
Prompt: A felicidade da sua vida
Original: A felicidade da sua vida depende da qualidade dos seus pensamentos.
Gerado: A felicidade da sua vida é o bem da sua felicidade. E o bem da sua felicidade é o bem da sua felicidade.
----------------------------------------------------------------------

[Frase 2]
Prompt: Não perca mais tempo discutindo
Original: Não perca mais tempo discutindo sobre o que um bom homem deve ser. Seja um.
Gerado: Não perca mais tempo discutindo a natureza do que é. O que está correto é a verdade. A verdade é o que está
----------------------------------------------------------------------

[Frase 3]
Prompt: Você tem poder sobre sua
Original: Você tem poder sobre sua mente, não sobre eventos externos. Perceba isso e encontrará a força.
Gerado: Você tem poder sobre sua vida e sobre o seu destino. Ele pode ser o único que pode salvar o mundo.


----------------------------------------------------------------------



---
## 13. Análise qualitativa: Vocabulário filosófico aprendido

In [85]:
# Prompts que testam vocabulário estoico
philosophy_prompts = [
    "A virtude é",
    "O destino",
    "A morte",
    "A natureza humana",
    "O tempo"
]

print("Análise de vocabulário filosófico estoico")
print("=" * 70)

for prompt in philosophy_prompts:
    base_out = generate_text(base_model, prompt, max_new_tokens=20, temperature=0.5)
    ft_out = generate_text(model, prompt, max_new_tokens=20, temperature=0.5)

    print(f"\nPrompt: '{prompt}'")
    print(f"  [BASE]: {base_out}")
    print(f"  [AJUSTADO]: {ft_out}")
    print("-" * 70)

Análise de vocabulário filosófico estoico

Prompt: 'A virtude é'
  [BASE]: A virtude é que o homem não pode ser considerado como um deus, mas como um deus.

Na mitologia
  [AJUSTADO]: A virtude é a certeza de que o bem não é afetado pela mal. A razão é a certeza de que o
----------------------------------------------------------------------

Prompt: 'O destino'
  [BASE]: O destino final de um jogo de futebol entre o Brasil e a Alemanha, a Copa do Mundo de 2014,
  [AJUSTADO]: O destino de todas as coisas depende do que elas fazem.

Para o bem, o bem é o
----------------------------------------------------------------------

Prompt: 'A morte'
  [BASE]: A morte de Carlos II, em 10 de novembro de 1383, marcou o fim do reinado de Carlos II
  [AJUSTADO]: A morte não é a morte, mas a morte de um homem. A morte de um homem é a morte
----------------------------------------------------------------------

Prompt: 'A natureza humana'
  [BASE]: A natureza humana é a única fonte de energia, e a úni

---
## Exercício Prático — Análise Comparativa

### Objetivo

Escolher prompts que **não estavam** nos dados de treino, mas que sejam do mesmo domínio (filosofia estoica). Comparar as saídas do modelo base e do modelo ajustado.

### Contexto do Dataset

**Marco Aurélio (121-180 d.C.)**
- Imperador romano e filósofo estoico
- Obra: "Meditações" (reflexões pessoais)
- Temas: virtude, natureza, destino, aceitação, razão, justiça

**Características do corpus:**
- 100 frases curtas e aforismos
- Linguagem reflexiva e contemplativa
- Foco em filosofia prática
- Vocabulário: alma, virtude, natureza, destino, sabedoria, tempo, morte

### Observações Esperadas

**Modelo Base (GPT-2 sem ajuste):**
- Gera texto mais genérico e moderno
- Pode usar linguagem informal ou coloquial
- Menos provável de usar termos filosóficos estoicos
- Estrutura mais variada e contemporânea

**Modelo Ajustado (após fine-tuning):**
- Adota tom reflexivo e contemplativo
- Usa vocabulário estoico (virtude, natureza, razão, destino)
- Estrutura similar aos aforismos de Marco Aurélio
- Temas recorrentes: aceitação, auto-controle, impermanência

### Vocabulário Técnico Aprendido

**Termos Filosóficos:**
- Virtude, sabedoria, justiça, temperança
- Natureza, universo, cosmos
- Destino, fortuna, providência
- Razão, logos, mente
- Alma, espírito, caráter

**Conceitos Estoicos:**
- Aceitação do que não pode ser mudado
- Controle sobre pensamentos e reações
- Impermanência e mudança
- Interconexão de todas as coisas
- Viver de acordo com a natureza

**Padrões Estruturais:**
- Sentenças declarativas curtas
- Imperativas (faça, seja, não faça)
- Metáforas naturais (rio, estrelas, abelha)
- Paralelismos e antíteses

### Análise dos Resultados

**Adaptação de Estilo:**
- Modelo aprende estrutura de aforismos
- Sentenças se tornam mais concisas e diretas
- Uso de linguagem prescritiva e reflexiva

**Temática Consistente:**
- Modelo mantém foco em temas estoicos
- Evita tangentes não-filosóficas
- Gera continuações coerentes com corpus

### Conclusão

O fine-tuning foi bem-sucedido em especializar o GPT-2 para filosofia estoica:

1. **Adaptação estilística**: Modelo adotou tom aforístico de Marco Aurélio
2. **Vocabulário específico**: Aprendeu termos filosóficos e estoicos
3. **Coerência temática**: Mantém foco em virtude, natureza e razão
4. **Eficiência**: Com apenas 100 frases e 10 épocas, observamos mudança significativa

**Limitações:**
- Dataset pequeno pode levar a overfitting
- Modelo pode memorizar frases exatas
- Diversidade limitada de expressões

**Recomendações para melhorar:**
- Expandir corpus com outras obras estoicas (Sêneca, Epiteto)
- Aumentar épocas gradualmente monitorando loss
- Usar data augmentation (paráfrase)
- Aplicar regularização (dropout, weight decay)

In [86]:
# Espaço para seus próprios testes

# Defina seus prompts aqui (no estilo estoico)
custom_prompts = [
    "A verdadeira paz vem de",
    "Aquele que busca a felicidade",
    "A vida justa requer"
]

print("TESTES PERSONALIZADOS")
print("=" * 70)

for prompt in custom_prompts:
    base = generate_text(base_model, prompt, max_new_tokens=30)
    finetuned = generate_text(model, prompt, max_new_tokens=30)

    print(f"\nPrompt: {prompt}")
    print("-" * 70)
    print(f"BASE:\n{base}\n")
    print(f"AJUSTADO:\n{finetuned}")
    print("=" * 70)

TESTES PERSONALIZADOS

Prompt: A verdadeira paz vem de
----------------------------------------------------------------------
BASE:
A verdadeira paz vem de uma fonte de energia. A paz é o resultado da amizade, da amizade e do respeito mútuo.

A paz não pode ser alcançada sem a

AJUSTADO:
A verdadeira paz vem de viver com Deus.

Para que o mundo possa estar melhor, não precisamos de nenhum tipo de luxo. Não precisamos de nada mais que um homem

Prompt: Aquele que busca a felicidade
----------------------------------------------------------------------
BASE:
Aquele que busca a felicidade e o triunfo, aquele que busca a perfeição, aquele que busca a felicidade e o triunfo, aquele que busca a perfeição, aquele que busca a perfeição

AJUSTADO:
Aquele que busca a felicidade não é nada além da felicidade. A felicidade é o que o homem tem para viver. A felicidade é a felicidade do homem.

O homem

Prompt: A vida justa requer
---------------------------------------------------------------------

---
## Salvando o modelo ajustado (opcional)

In [87]:
# Salvar modelo e tokenizador
output_dir = "./gpt2_marco_aurelio"

model.save_pretrained(output_dir)
tok.save_pretrained(output_dir)

print(f"Modelo salvo em: {output_dir}")
print("\nPara carregar depois:")
print(f"  model = AutoModelForCausalLM.from_pretrained('{output_dir}')")
print(f"  tokenizer = AutoTokenizer.from_pretrained('{output_dir}')")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Modelo salvo em: ./gpt2_marco_aurelio

Para carregar depois:
  model = AutoModelForCausalLM.from_pretrained('./gpt2_marco_aurelio')
  tokenizer = AutoTokenizer.from_pretrained('./gpt2_marco_aurelio')
